# Sample RAG Agent Walkthrough

This notebook will walk users through setting up a sample RAG Agent and running it against the Hugging Face 'rag-mini-wikipedia' dataset (https://huggingface.co/datasets/rag-datasets/rag-mini-wikipedia)

#### Ensure the latest version of boto3 is shown below

##### If not then run through setup_environment.ipynb in the 0-Notebook-environment/ folder

In [27]:
!pip freeze | grep boto3

boto3==1.37.2


#### Load in environment variables to notebook

In [28]:
# Retrieve import path
%store -r IMPORTS_PATH

# Retrieve account info
%store -r account_id
%store -r region

# Retrieve model lists
%store -r agent_foundation_model

#### Retrieve imports environment variable and bring libraries into notebook

In [29]:
%run $IMPORTS_PATH

Successfully imported necessary libraries into notebook


# Prerequisites

This notebook assumes that you have gone through the notebook environment setup in the 0-Notebook-environment/ folder and have set up a Langfuse project

# Data Preparation

### Retrieve Hugging Face 'rag-mini-wikipedia'

In [30]:
# Retrieves dataset corpus using Datasets Python library

ds_corpus = load_dataset("rag-datasets/rag-mini-wikipedia", "text-corpus")

### Write text corpus to file and upload to Amazon S3 to use as data source for knowledge base

In [31]:
# Write whole corpus to a .txt file

with open('mini_wiki.txt', 'w') as f:
    f.write(str(ds_corpus['passages']['passage']))

print("You can now view the whole Wikipedia corpus in mini_wiki.txt")

You can now view the whole Wikipedia corpus in mini_wiki.txt


In [32]:
# Create Amazon S3 bucket and upload .txt. file to Amazon S3 bucket

s3_client = boto3.client('s3')

wiki_bucket_name = f"rag-mini-wiki-{account_id}-{uuid.uuid4().hex[:6]}"

s3_client.create_bucket(
    Bucket=wiki_bucket_name,
    CreateBucketConfiguration={
        'LocationConstraint': region
    }
)

%store wiki_bucket_name

print("Created bucket with name '{}' in region '{}'".format(wiki_bucket_name, region))

Stored 'wiki_bucket_name' (str)
Created bucket with name 'rag-mini-wiki-861276117215-fba410' in region 'us-west-2'


In [33]:
# Place .txt corpus in S3 bucket

s3_client.upload_file('mini_wiki.txt', wiki_bucket_name, 'mini_wiki.txt')

print("Uploaded corpus to '{}'".format(wiki_bucket_name))

Uploaded corpus to 'rag-mini-wiki-861276117215-fba410'


### Create Bedrock Knowledge Base
#### Follow the steps below to create a Bedrock Knowledge Base in the AWS Console manually

Step 1: Navigate to the 'Amazon Bedrock' service in the AWS Console and navigate to the 'Knowledge Bases' section

Step 2: Click 'Create' and select 'Knowledge Base with vector store'

Step 3: Name the Knowledge Base 'mini-wiki-kb' and select the Amazon S3 data source radio button

Step 4: Name the data source 'mini-wiki-data' and select the S3 bucket file 'mini_wiki.txt' that was uploaded, 
        e.x. s3://rag-mini-wikipedia-data-XXXXXXXXXXXX/mini_wiki.txt

Step 5: Use the default parsing and default chunking options

Step 6: Select the 'Titan Text Embeddings V2' embedding model and create an Amazon OpenSearch Serverless vector store with the quick create option

Step 7: Now create the knowledge base

Step 8: Manually sync the data source with the knowledge base by clicking on the data source and selecting 'Sync' and wait for the process to finish before proceeding to the next step

In [37]:
# Fetch knowledge base ID

bedrock_agent_client = boto3.client("bedrock-agent", region)

# Call the list_knowledge_bases method
response = bedrock_agent_client.list_knowledge_bases()
wiki_kb_id = None

# Iterate through knowledge bases and find needed one
if 'knowledgeBaseSummaries' in response:
    for kb in response['knowledgeBaseSummaries']:
        if 'mini-wiki-kb' in kb['name']:
            wiki_kb_id = kb['knowledgeBaseId']

%store wiki_kb_id

wiki_kb_id

Stored 'wiki_kb_id' (str)


'X6FJE1ZRYL'

### Create RAG Agent

In [34]:
agent_name = 'sample-rag-agent'
agent_description = "RAG agent to run against the Hugging Face 'rag-mini-wikipedia' dataset"
agent_instruction = """Use the associated knowledge base to answer questions."""

In [35]:
agents = AgentsForAmazonBedrock()

rag_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False,
    verbose=False
)

rag_agent

('DS8G3449MI',
 'TSTALIASID',
 'arn:aws:bedrock:us-west-2:861276117215:agent-alias/DS8G3449MI/TSTALIASID')

In [36]:
rag_agent_id = rag_agent[0]
rag_agent_arn = f"arn:aws:bedrock:{region}:{account_id}:agent/{rag_agent_id}"

rag_agent_id, rag_agent_arn

('DS8G3449MI', 'arn:aws:bedrock:us-west-2:861276117215:agent/DS8G3449MI')

In [38]:
agents.associate_kb_with_agent(
    rag_agent_id,
    "Hugging Face 'rag-mini-wikipedia' dataset", 
    wiki_kb_id
)

### Test RAG Agent

#### Invoke Sample RAG Agent Test Alias to see that it answers question properly

In [39]:
# Ask example question to agent

bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region)

session_id:str = str(uuid.uuid1())

test_query = "Who suggested Lincoln grow a beard?"
response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=rag_agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)

print("Request sent to Agent")
print("====================")
print("Agent processing query now")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")

Request sent to Agent
Agent processing query now
Agent Answer: Grace Bedell, an 11-year-old girl, suggested to Lincoln that he grow a beard in 1860.


#### Prepare agent and create alias for use with evaluation framework

In [40]:
rag_agent_alias_id, rag_agent_alias_arn = agents.create_agent_alias(
    rag_agent[0], 'v1'
)

%store rag_agent_alias_arn
rag_agent_alias_id, rag_agent_alias_arn

Stored 'rag_agent_alias_arn' (str)


('K72I4ACIFL',
 'arn:aws:bedrock:us-west-2:861276117215:agent-alias/DS8G3449MI/K72I4ACIFL')

### Create input .json file for all RAG using ground truth provided by Hugging Face dataset

#### Below is the option to specify the number of question to generate. 

#### Default is 10, set to -1 to run through all questions, or specify to any other desired number

In [42]:
number_questions = -1

In [43]:
# Create input json data file for evaluation framework and place so it can be run by evaluation framework by user

ds_qa = load_dataset("rag-datasets/rag-mini-wikipedia", "question-answer")

input_data_dict = {}

# Iterate through all elements in dataset
for index, data in enumerate(ds_qa['test']):

    # Extract desired number of questions
    if number_questions != -1:
        if index == number_questions:
            break

    qa_pair = {
        "question_id": data['id'],
        "question_type": "RAG",
        "question": data['question'],
        "ground_truth": data['answer']
    }   
    input_data_dict["Trajectory{}".format(index)] = [qa_pair]

# Save to JSON file
with open('rag_data_file_auto.json', 'w', encoding='utf-8') as f:
    json.dump(input_data_dict, f, indent=4, ensure_ascii=False)

## Run Bedrock Agent Evaluation Framework on the newly created sample RAG agent

Step 1: Navigate to config_tpl.py at the root of the repository and create a copy named 'config.py'

Step 2: Fill in config.py with the below information

In [41]:
print("AGENT_ID='{}'".format(rag_agent_id))
print("AGENT_ALIAS_ID='{}'".format(rag_agent_alias_id))
print("DATA_FILE_PATH='{}'".format("blog_sample_agents/1-Sample-rag-agent/rag_data_file_auto.json"))

AGENT_ID='DS8G3449MI'
AGENT_ALIAS_ID='K72I4ACIFL'
DATA_FILE_PATH='blog_sample_agents/1-Sample-rag-agent/rag_data_file_auto.json'


Step 3: Input the keys provided by your Langfuse project into the 'config.py' file

Step 4: Run the evaluation framework against the dataset by opening a terminal, navigate to the root of the repository, and run 'python3 driver.py' 

Step 5: Look at your Langfuse project's dashboard and traces to see the evaluation results!